In [5]:
from Node import DS
from Tools import Func
from Tools import Config
import pandas as pd
import numpy as np
import os
import time
import operator
import sys
import re

# ==========================UpdateSk==========================
def UpdateSk(element,Sk_head,Sk):
    e_max=Config.e_max
    col,row=Func.position(element)
        # col / row index of element 
    #print("{} send to Sk[{}][{}]".format(element,row,col))
    # ==========================update sketch==========================
    Sk_head[row].count+=element.count
    Sk_head[row].distinct.add(element.ID)
    Sk[row][col]+=1
    Update_local_max(Sk_head[row],Sk[row],element,col)
    Update_emax(Sk_head,Sk,row)

# ==========================update local max==========================       
def Update_local_max(head_item,element_list,element,column):
    # local max need only 1 row
    #print("In Update_local_max:")
    width,depth=Config.width,Config.depth
    if head_item.maxID=='':
        head_item.maxID=element.ID
        head_item.keep=element.count
    elif head_item.maxID==element.ID:
        head_item.keep+=1
    else:
        # head_item.maxID!=element.ID:
        local_max_col,local_max_row=Func.position(DS.Tail(head_item.maxID,0))
        if local_max_col==column:
            # local_max collision with e
            head_item.keep-=element.count
            if head_item.keep<0:
                head_item.maxID=element.ID
                head_item.keep=element.count
        elif element_list[local_max_col]<element_list[column]:
            # no collision
            head_item.maxID=element.ID
            head_item.keep=element.count       
        if element_list[column]>element_list[local_max_col]:
            head_item.maxID=element.ID   
        '''
        # head_item.maxID!=element.ID:
        local_max_col,local_max_row=Func.position(DS.Tail(head_item.maxID,0))
        if local_max_col==column:
            # local_max collision with e
            head_item.keep-=element.count
            if head_item.keep<0:
                head_item.maxID=element.ID
                head_item.keep=element.count
        elif element_list[local_max_col]<element_list[column]:
            # no collision
            head_item.maxID=element.ID
            head_item.keep=element.count       
   
            
            
        local_max_col,local_max_row=Func.position(DS.Tail(head_item.maxID,0))
        if element_list[column]>element_list[local_max_col]:
            head_item.maxID=element.ID                
        '''        


# ==========================update e_max==========================
def Update_emax(head,sketch,sk_row):
    local_max_col,local_max_row=Func.position(DS.Tail(head[sk_row].maxID,0))
    if sketch[local_max_row][local_max_col]>Config.e_max.count:
        Config.e_max.ID=head[sk_row].maxID
        Config.e_max.count=sketch[local_max_row][local_max_col]

# ========================== BringBack=========================
def BringBack(e_min,head,sketch):
    # bring e_max back to Top
    # e_min=e_max, e_max=Null, delete e_max.count in Sketch, send e_min into Sketch
    temp=DS.Tail(e_min.ID,e_min.count)
    e_min.ID=Config.e_max.ID
    e_min.count=Config.e_max.count
    c,r=Func.position(Config.e_max)
    head[r].bringback+=1
    DeleteSk(e_max,head,sketch)
    UpdateSk(temp,head,sketch)

# ==========================DeleteSk=========================
def DeleteSk(element,head,sketch):
    # e_max in sketch: sketch[r][c]=0, total count-=sketch[row][col]
    #width,depth=get_width_depth()
    col,row=Func.position(element)
    head[row].count-=element.count
        # total_count-=element.count
    sketch[row][col]=0
    head[row].maxID=''
    element.ID=""
    element.count=0

    
# ==========================main=========================    
dataset='webdocs'
datapath='..\dataset\webdocs'
pattern='out_.*'
r=re.compile(pattern)
filelist=list(filter(r.match,os.listdir(datapath)))

w=256
d=128
size=512
Config.Set_default(w,d,size)
    # set width, depth, size of Sk, random seed of hash
    # Config.width, Config.depth
e_max=Config.e_max
    # initialize e_max
    
Sk_head=[DS.Head(0) for j in range(Config.depth)]
Sketch=np.zeros((Config.depth,Config.width),dtype='int32')

Top=[]

#item_count=100
#income=0
start=time.time()
for datafile in filelist[:1]:
    src_data=os.path.join(datapath,datafile)
    with open(src_data,'r') as file:
        while True:
            e=file.readline().strip('\n')
            if not e:
                print("EOF")
                break
            else:
            # item_count-=1
            #income+=1
            #print("read {}-th element:{}".format(income,e))
                item=DS.Tail(e,1)
                index=Func.find(item,Top)
                if index<0:
                    if len(Top)<Config.size:
                        Top.append(item)
                        # 此處的count必等於1，不用sorting
                        index=len(Top)-1
                    else:
                        UpdateSk(item,Sk_head,Sketch)
                else:
                    Top[index].count+=1
                    if index==0 or Top[index].count< Top[index-1].count:
                        pass
                    else:
                        Top.sort(key=operator.attrgetter('count'),reverse=True)                
            if Config.e_max.count>Top[len(Top)-1].count:
                BringBack(Top[len(Top)-1],Sk_head,Sketch)
                Top.sort(key=operator.attrgetter('count'),reverse=True)
                #print('Top after BringBack: \n\t{}'.format(Top)) 

end=time.time()
print("Top-{},Sketch:{}*{}".format(Config.size,Config.depth,Config.width))
print("Execution time:{:8.3f} seconds.".format(end-start))
print("Total memory {} bytes=".format(sys.getsizeof(Top)+Sketch.nbytes+sys.getsizeof(Sk_head[0])*Config.depth),end='')
print("Top:{} bytes, Sketch:{} bytes, Sketch_head:{} bytes.".format(sys.getsizeof(Top),Sketch.nbytes,sys.getsizeof(Sk_head[0])*Config.depth))

#====================Top to csv=============================
my_csv_file=Func.Top_to_csv(Top,dataset)
    # return "My_OO_kosarak_Top_512_128_256.csv"

#====================result compare=============================
# precision
gr_dir=r"..\dataset\webdocs"
gr_file_name='webdocs_00_ground_truth.csv'
gr_path=os.path.join(gr_dir,gr_file_name)
    # gr_path="..\dataset\webdocs\webdocs_00_ground_truth.csv"
my_result_path=os.path.join(r'..\result',my_csv_file)
    # my_result_path="..\result\My_OO_kosarak_Top_512_128_256.csv"
true_positive_set,precision=Func.Get_precision(gr_path,my_result_path)

# ====================ARE/AAE for top and all====================
startx=time.time()
top_are,top_aae,all_are,all_aae,tp,fp=Func.Get_ARE_AAE(gr_path,my_result_path,Top,Sk_head,Sketch,2)
endx=time.time()
print("Query2:")
print("Find:{}, TP:{}, FP:{}".format(len(true_positive_set),tp,fp))
print("Top_ARE: {:8.6f}".format(top_are))
print("Top_AAE: {:8.6f}".format(top_aae))
print("all_ARE: {:8.6f}".format(all_are))
print("all_AAE: {:8.6f}".format(all_aae))
print("Estimate time:{:8.3f} seconds.".format(endx-startx))


'''

startx=time.time()
top_are,top_aae,all_are,all_aae,tp,fp=Func.Get_ARE_AAE(gr_path,my_result_path,Top,Sk_head,Sketch,1)
endx=time.time()

print("Query1:")
print("Find:{}, TP:{}, FP:{}".format(len(true_positive_set),tp,fp))
print("Top_ARE: {:8.6f}".format(top_are))
print("Top_AAE: {:8.6f}".format(top_aae))
print("all_ARE: {:8.6f}".format(all_are))
print("all_AAE: {:8.6f}".format(all_aae))
print("Estimate time:{:8.3f} seconds.".format(endx-startx))

startx=time.time()
top_are,top_aae,all_are,all_aae,tp,fp=Func.Get_ARE_AAE(gr_path,my_result_path,Top,Sk_head,Sketch,3)
endx=time.time()
print("Query3:")
print("Find:{}, TP:{}, FP:{}".format(len(true_positive_set),tp,fp))
print("Top_ARE: {:8.6f}".format(top_are))
print("Top_AAE: {:8.6f}".format(top_aae))
print("all_ARE: {:8.6f}".format(all_are))
print("all_AAE: {:8.6f}".format(all_aae))
print("Estimate time:{:8.3f} seconds.".format(endx-startx))

startx=time.time()
top_are,top_aae,all_are,all_aae,tp,fp=Func.Get_ARE_AAE(gr_path,my_result_path,Top,Sk_head,Sketch,4)
endx=time.time()
print("Query4:")
print("Find:{}, TP:{}, FP:{}".format(len(true_positive_set),tp,fp))
print("Top_ARE: {:8.6f}".format(top_are))
print("Top_AAE: {:8.6f}".format(top_aae))
print("all_ARE: {:8.6f}".format(all_are))
print("all_AAE: {:8.6f}".format(all_aae))
print("Estimate time:{:8.3f} seconds.".format(endx-startx))

'''

# ====================Cover result into a dataframe====================
result_para=(startx,endx,true_positive_set,precision,top_are,top_aae,all_are,all_aae,tp,fp)
df=Func.To_df(Top,Sketch,Sk_head,*result_para)
df

EOF
Top-512,Sketch:128*256
Execution time: 984.498 seconds.
Total memory 142512 bytes=Top:4272 bytes, Sketch:131072 bytes, Sketch_head:7168 bytes.
My_OO_webdocs_Top_512_128_256.csv
Precision:   0.9434
Query2:
Find:483, TP:483, FP:29
Top_ARE: 0.008459
Top_AAE: 57.865424
all_ARE: 2.169822
all_AAE: 16.912976
Estimate time:  35.010 seconds.


,Top-k,Sketch,Total memory,Exe_time,Find,Precision,ARE-Top,AAE-Top,ARE-all,AAE-all
0,512,128*256,142512 bytes =Top:4272+ Sketch:131072+ Sk_head...,35.01,"Find:483, TP:483, FP:29",0.9434,0.008459,57.865424,2.169822,16.912976


In [2]:
EOF
Top-512,Sketch:128*256
Execution time: 965.150 seconds.
Total memory 142512 bytes=Top:4272 bytes, Sketch:131072 bytes, Sketch_head:7168 bytes.
My_OO_webdocs_Top_512_128_256.csv
Precision:   0.9414
Query2:
Find:482, TP:482, FP:30
Top_ARE: 0.007596
Top_AAE: 52.877593
all_ARE: 3.358231
all_AAE: 18.423094
Estimate time:  34.113 seconds.

,Top-k,Sketch,Total memory,Exe_time,Find,Precision,ARE-Top,AAE-Top,ARE-all,AAE-all
0,512,128*256,142512 bytes =Top:4272+ Sketch:131072+ Sk_head...,34.113,"Find:482, TP:482, FP:30",0.9414,0.007596,52.877593,3.358231,18.423094


In [6]:
gr_dir=r"..\dataset\webdocs"
gr_file_name='webdocs_00_ground_truth.csv'
grtruth=pd.read_csv(os.path.join(gr_dir,gr_file_name))

#print([i.count for i in Top])
print("sum of ground truth Top[{}]:{}".format(Config.size,grtruth['Count'][:512].sum()))
print("sum of Top[512] in algo:{}".format(sum([i.count for i in Top])))
print("sum of Sketch:{}".format(Sketch.sum()))
print("sum of Sk_head:{}".format(sum([j.count for j in Sk_head])))
print("sum of all count of algo:{}".format(sum([i.count for i in Top])+Sketch.sum()))
print("sum of Top + Sk_head:{}".format(sum([i.count for i in Top])+sum([j.count for j in Sk_head])))

grtruth['Count'].sum()

sum of ground truth Top[512]:5900071
sum of Top[512] in algo:5852240
sum of Sketch:9121300
sum of Sk_head:9635654
sum of all count of algo:14973540
sum of Top + Sk_head:15487894


15487894

In [ ]:
Top-512,Sketch:128*256
Execution time:1179.328 seconds.
Total memory 142512 bytes=Top:4272 bytes, Sketch:131072 bytes, Sketch_head:7168 bytes.
My_OO_webdocs_Top_512_128_256.csv
Precision:   0.9492

In [ ]:
from Node import DS
from Tools import Config
import pandas as pd
import os
import spookyhash
import mmh3
import sys


# ==========================Tools=========================  


def find(e,element_list):
    # return index of e in element_list
    try:
        index=[ele.ID for ele in element_list].index(e.ID)
    except:
        index=-99
    return index

def position(element):
    hash1=mmh3.hash(element.ID,seed=Config.seed[0], signed=False)
    hash2=mmh3.hash(element.ID,seed=Config.seed[1], signed=False)
    col=hash1 % Config.width
    row=hash2 % Config.depth
    return col,row 

def Top_to_csv(Top,dataset_name):
    templi=[[i.ID,i.count] for i in Top]
    df=pd.DataFrame(templi,columns=['ID', 'Count'])
    df=df.sort_values(by='Count',ascending=False)
    name="My_OO_"+dataset_name+"_Top"+'_'+str(Config.size)+'_'+str(Config.depth)+'_'+str(Config.width)+'.csv'
    # My_OO_kosarak_Top_size_d_w.csv" for algorithm result
    df.to_csv(os.path.join(r'..\result',name),index=False)
    print(name)
    return name
    
def Get_precision(ground_truth_path,result_path):
    #gr_truth,result: path of csv file
    grtruth=pd.read_csv(ground_truth_path)
        # ground truth of whole data set, [Element, Count]
    My_result=pd.read_csv(result_path)
        # Top-k of algorithm result
    # precision
    gt_set=set(grtruth['Element'][:Config.size])
        # e.ID only
    my_set=set(My_result['ID'])
    tp_set=gt_set & my_set
        # true-positive set
        # &: set 交集運算
    precision=len(tp_set)/len(my_set)
        # &: set 交集運算
    print("Precision: {:8.4f}".format(precision))
    return tp_set,precision

def Get_ARE_AAE(ground_truth_path,result_path,Top,Sk_head,Sketch,true_positive_set,method):
    grtruth=pd.read_csv(ground_truth_path)
        # DF of ground truth of whole data set
    My_result=pd.read_csv(result_path)
        # DF of algorithm result
    gt_dict=dict(grtruth.values.tolist())
    top_dict=dict(My_result.values.tolist())
    distinct=len(gt_dict)
        # cardinality of all incoming elements
    row_cardinality=[len(i.distinct) for i in Sk_head]
        # number of distinct elements in each row in Sk_head
    LHH_list=[i.maxID for i in Sk_head]
        # list of local max elements in each row of Sketch    
    all_are=0
    all_aae=0
    top_are=0
    top_aae=0
    tp=0
    fp=0
    
    for item in gt_dict:
        if method==1:
            count=Query(str(item),Top,LHH_list,Sketch,row_cardinality)
        elif method==2:
            count=Query2(str(item),Top,Sk_head,Sketch,row_cardinality)
        elif method==3:
            count=Query3(str(item),Top,Sk_head,Sketch,row_cardinality)
        elif method==4:
            count=Query4(str(item),Top,Sk_head,Sketch,row_cardinality)            
        all_are+=abs(count-gt_dict[item])/gt_dict[item]
        all_aae+=abs(count-gt_dict[item])
        if item in top_dict:
            # compute ARE/AAE of all elements in Top but not only hit elements
            top_are+=abs(count-gt_dict[item])/gt_dict[item]
            top_aae+=abs(count-gt_dict[item])
            if item in true_positive_set:
                tp+=1
            else:
                fp+=1
    top_are=top_are/Config.size
    top_aae=top_aae/Config.size
    all_are=all_are/distinct
    all_aae=all_aae/distinct        
    return top_are,top_aae,all_are,all_aae,tp,fp

def To_df(Top,Sketch,Sk_head,*args):
    (startx,endx,true_positive_set,tp,fp,precision,top_are,top_aae,all_are,all_aae,tp,fp)=args
    sketch_size=str(Config.depth)+'*'+str(Config.width)
    temp=sys.getsizeof(Top)+Sketch.nbytes+sys.getsizeof(Sk_head[0])*Config.depth
    memory_usage=str(temp)+' bytes ='+'Top:'+str(sys.getsizeof(Top))+'+ Sketch:'+str(Sketch.nbytes)+'+ Sk_head:'+str(sys.getsizeof(Sk_head[0])*Config.depth)

    result_df=pd.DataFrame(columns=['Top-k',
                                    'Sketch',
                                    'Total memory',
                                    'Exe_time',
                                    'Find',
                                    'Precision',
                                    'ARE-Top',
                                    'AAE-Top',
                                    'ARE-all',
                                    'AAE-all'])
    output_dict={
        'Top-k':Config.size,
        'Sketch':sketch_size,
        'Total memory':memory_usage,
        'Exe_time':float('{:.3f}'.format(endx-startx)),
        'Find':"Find:{}, TP:{}, FP:{}".format(len(true_positive_set),tp,fp),
        'Precision':float("{:8.4f}".format(precision)),
        'ARE-Top':float('{:8.6f}'.format(top_are)),
        'AAE-Top':float('{:8.6f}'.format(top_aae)),
        'ARE-all':float('{:8.6f}'.format(all_are)),
        'AAE-all':float('{:8.6f}'.format(all_aae))}

    result_df=result_df.append(output_dict,ignore_index=True)
    file="My_OO_kosarak_result"+'_'+str(Config.size)+'_'+str(Config.depth)+'_'+str(Config.width)+'_.csv'
    result_df.to_csv(file,index=False)
    return result_df

        
def Query(e,Top,local_maxID,Sketch,row_cardinality):
    # e is ID, type(e)=str
    # row_cardinality is list of number of distinct elements in each row in Sk_head
    # local_maxID is list of local max element in each row of Sketch
    count=0
    index=find(DS.Tail(e,0),Top)
    if index>=0:
        # e in Top
        count=Top[index].count
    else:
        # e in Sketch
        col,row=position(DS.Tail(e,0))
        if e in local_maxID:
            # e is LHH
            count=Sketch[row][col]
        else:
            # e is not LHH
            ratio=Config.width/row_cardinality[row]
            count=int(Sketch[row][col]*ratio)
    if count<=1:
        count=1
    return count    
    
    
def Query2(e,Top,Sk_head,Sketch,row_cardinality):
    # e is ID, type(e)=str
    # row_cardinality is list of number of distinct elements in each row in Sk_head
    # local_maxID is list of local max element in each row of Sketch
    count=0
    index=find(DS.Tail(e,0),Top)
    if index>=0:
        # e in Top
        count=Top[index].count
    else:
        # e in Sketch
        col,row=position(DS.Tail(e,0))
        ratio=Config.width/row_cardinality[row]
        if e == Sk_head[row].maxID:
            # e is LHH
            count=Sk_head[row].keep+int((Sketch[row][col]-Sk_head[row].keep)*ratio)
        else:
            # e is not LHH
            count=int((Sketch[row][col]-Sk_head[row].keep)*ratio)
    if count<=1:
        count=1
    return count

def Query3(e,Top,Sk_head,Sketch,row_cardinality):
    # e is ID, type(e)=str
    # row_cardinality is list of number of distinct elements in each row in Sk_head
    # local_maxID is list of local max element in each row of Sketch
    count=0
    index=find(DS.Tail(e,0),Top)
    if index>=0:
        # e in Top
        count=Top[index].count
    else:
        # e in Sketch
        col,row=position(DS.Tail(e,0))
        ratio=Config.width/(row_cardinality[row]-Sk_head[row].bringback)
        if e == Sk_head[row].maxID:
            # e is LHH
            count=Sk_head[row].keep+int((Sketch[row][col]-Sk_head[row].keep)*ratio)
        else:
            # e is not LHH
            lhh_col,lhh_row=position(DS.Tail(Sk_head[row].maxID,0))
            if col==lhh_col:
                count=int((Sketch[row][col]-Sk_head[row].keep)*ratio)
            else:
                count=int(Sketch[row][col]*ratio)
    if count<=1:
        count=1
    return count

def Query4(e,Top,Sk_head,Sketch,row_cardinality):
    # e is ID, type(e)=str
    # row_cardinality is list of number of distinct elements in each row in Sk_head
    # local_maxID is list of local max element in each row of Sketch
    count=0
    index=find(DS.Tail(e,0),Top)
    if index>=0:
        # e in Top
        count=Top[index].count
    else:
        # e in Sketch
        col,row=position(DS.Tail(e,0))
        ratio=Config.width/(row_cardinality[row]-Sk_head[row].bringback)
        if e == Sk_head[row].maxID:
            # e is LHH
            count=Sk_head[row].keep+int((Sketch[row][col]-Sk_head[row].keep)*ratio)
        else:
            # e is not LHH
            count=int(Sketch[row][col]*ratio)
    if count<=1:
        count=1
    return count

def Get_ARE_AAE2(ground_truth_path,result_path,Top,Sk_head,Sketch,true_positive_set,method):
    grtruth=pd.read_csv(ground_truth_path)
        # DF of ground truth of whole data set
    My_result=pd.read_csv(result_path)
        # DF of algorithm result
    gt_dict=dict(grtruth.values.tolist())
    top_dict=dict(My_result.values.tolist())
    distinct=len(gt_dict)
        # cardinality of all incoming elements
    row_cardinality=[len(i.distinct) for i in Sk_head]
        # number of distinct elements in each row in Sk_head
    LHH_list=[i.maxID for i in Sk_head]
        # list of local max elements in each row of Sketch    
    all_are=0
    all_aae=0
    top_are=0
    top_aae=0
    tp=0
    fp=0
    top_smaller=0
    top_bigger=0
    smaller=0
    bigger=0
    top_error=0
    other_error=0
    all_count=0
    for item in gt_dict:
        if method==1:
            count=Query(str(item),Top,LHH_list,Sketch,row_cardinality)
        elif method==2:
            count=Query2(str(item),Top,Sk_head,Sketch,row_cardinality)
        elif method==3:
            count=Query3(str(item),Top,Sk_head,Sketch,row_cardinality)
        elif method==4:
            count=Query4(str(item),Top,Sk_head,Sketch,row_cardinality)            
        all_are+=abs(count-gt_dict[item])/gt_dict[item]
        all_aae+=abs(count-gt_dict[item])
        if item in top_dict:
            # compute ARE/AAE of all elements in Top but not only hit elements
            if count <gt_dict[item]:
                top_error+=abs(count-gt_dict[item])
                top_smaller+=1
                #print("ID:{},count:{},truth:{}".format(item,count,gt_dict[item]))
                if (gt_dict[item]/count)>1.05:
                    print("ID:{},count:{},truth:{}".format(item,count,gt_dict[item]))
            elif count >gt_dict[item]:
                top_error+=abs(count-gt_dict[item])
                top_bigger+=1
                #print("ID:{},count:{},truth:{}".format(item,count,gt_dict[item]))
                if (count/gt_dict[item])>1.05:
                    print("ID:{},count:{},truth:{}".format(item,count,gt_dict[item]))            
            top_are+=abs(count-gt_dict[item])/gt_dict[item]
            top_aae+=abs(count-gt_dict[item])
            if item in true_positive_set:
                tp+=1
            else:
                fp+=1
        else:
            if count <gt_dict[item]:
                other_error+=abs(count-gt_dict[item])
                smaller+=1
            elif count >gt_dict[item]:
                other_error+=abs(count-gt_dict[item])
                bigger+=1 
    top_are=top_are/Config.size
    top_aae=top_aae/Config.size
    all_are=all_are/distinct
    all_aae=all_aae/distinct
    print("smaller:{},bigger:{}".format(smaller,bigger))
    print("top_smaller:{},top_bigger:{}".format(top_smaller,top_bigger))
    print("top_error:{}, other_error:{}".format(top_error,other_error))
    return top_are,top_aae,all_are,all_aae,tp,fp


Get_ARE_AAE2(gr_path,my_result_path,Top,Sk_head,Sketch,true_positive_set,3)

In [ ]:
all_count=0
for i in Top:
    all_count+=i.count
all_count+=Sketch.sum()
print(all_count)

In [ ]:
gr_path='..\dataset\webdocs'
gr_file_name='webdocs_00_ground_truth.csv'
grtruth=pd.read_csv(os.path.join(gr_path,gr_file_name))
grtruth['Count'].sum()

In [ ]:
all_count=0
for i in Top:
    all_count+=i.count
for j in Sk_head:
    all_count+=j.count
all_count

In [ ]:
Sketch.sum()

In [ ]:
print([i.count for i in Top])